# Homework 13 - Network Compression

Author: Liang-Hsuan Tseng (b07502072@ntu.edu.tw), modified from ML2021-HW13

If you have any questions, feel free to ask: ntu-ml-2022spring-ta@googlegroups.com

[**Link to HW13 Slides**](https://docs.google.com/presentation/d/1nCT9XrInF21B4qQAWuODy5sonKDnpGhjtcAwqa75mVU/edit#slide=id.p)

## Outline

* [Packages](#Packages) - intall some required packages.
* [Dataset](#Dataset) - something you need to know about the dataset.
* [Configs](#Configs) - the configs of the experiments, you can change some hyperparameters here.
* [Architecture_Design](#Architecture_Design) - depthwise and pointwise convolution examples and some useful links.
* [Knowledge_Distillation](#Knowledge_Distillation) - KL divergence loss for knowledge distillation and some useful links.
* [Training](#Training) - training loop implementation modified from HW3.
* [Inference](#Inference) - create submission.csv by using the student_best.ckpt from the previous experiment.



In [ ]:
from google.colab import drive
import os
# 挂载Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchsummary

### Packages
First, we need to import some useful packages. If the torchsummary package are not intalled, please install it via `pip install torchsummary`

In [ ]:
# Import some useful packages for this homework
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset # "ConcatDataset" and "Subset" are possibly useful
from torchvision.datasets import DatasetFolder, VisionDataset
from torchsummary import summary
from tqdm.auto import tqdm
import random

# !nvidia-smi # list your current GPU

### Configs
In this part, you can specify some variables and hyperparameters as your configs.

In [ ]:
cfg = {
    'dataset_root': '/content/drive/MyDrive/food11-hw13',
    'save_dir': '/content/drive/MyDrive/outputs',           # 输出保存目录
    'exp_name': "strong_baseline",     # 实验名称
    'batch_size': 64,                 # 批次大小128
    'lr': 1.4e-3,                       # 学习率3e-3
    'seed': 20220013,                 # 随机种子
    'loss_fn_type': 'KD',             # 损失函数类型：KD表示知识蒸馏
    'weight_decay': 5e-5,             # 权重衰减（L2正则化）
    'grad_norm_max': 10,              # 梯度裁剪的最大范数
    'n_epochs': 150,                  # 训练轮数
    'patience': 20,                   # 早停法的耐心值
}

In [ ]:
myseed = cfg['seed']  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
random.seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

save_path = os.path.join(cfg['save_dir'], cfg['exp_name']) # create saving directory
os.makedirs(save_path, exist_ok=True)

# define simple logging functionality
log_fw = open(f"{save_path}/log.txt", 'w') # open log file to save log outputs
def log(text):     # define a logging function to trace the training process
    print(text)
    log_fw.write(str(text)+'\n')
    log_fw.flush()

log(cfg)  # log your configs to the log file

{'dataset_root': '/content/drive/MyDrive/food11-hw13', 'save_dir': './outputs', 'exp_name': 'strong_baseline', 'batch_size': 64, 'lr': 0.0014, 'seed': 20220013, 'loss_fn_type': 'KD', 'weight_decay': 5e-05, 'grad_norm_max': 10, 'n_epochs': 150, 'patience': 20}


### Dataset
We use Food11 dataset for this homework, which is similar to homework3. But remember, Please DO NOT utilize the dataset of HW3. We've modified the dataset, so you should only access the dataset by loading it in this kaggle notebook or through the links provided in the HW13 colab notebooks.

In [ ]:
# fetch and download the dataset from github (about 1.12G)
# !wget https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW13/food11-hw13.tar.gz
## backup links:

#!wget https://github.com/andybi7676/ml2022spring-hw13/raw/main/food11-hw13.tar.gz -O food11-hw13.tar.gz
# !gdown '1ijKoNmpike_yjUw8SWRVVWVoMOXXqycj' --output food11-hw13.tar.gz

In [ ]:
# extract the data
#!tar -xzf ./food11-hw13.tar.gz # Could take some time
# !tar -xzvf ./food11-hw13.tar.gz # use this command if you want to checkout the whole process.

In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/food11-hw13'):
    if len(filenames) > 0:
        print(f"{dirname}: {len(filenames)} files.") # Show the file amounts in each split.

/content/drive/MyDrive/food11-hw13: 1 files.
/content/drive/MyDrive/food11-hw13/training: 9866 files.
/content/drive/MyDrive/food11-hw13/validation: 3431 files.
/content/drive/MyDrive/food11-hw13/evaluation: 3347 files.


Next, specify train/test transform for image data augmentation.
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms. You can also apply the knowledge or experience you learned in HW3.

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# define training/testing transforms
test_tfm = transforms.Compose([
    # It is not encouraged to modify this part if you are using the provided teacher model. This transform is stardard and good enough for testing.
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

train_tfm = transforms.Compose([
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1, hue=0.05),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.4),
    transforms.RandomAffine(
        degrees=(-20, 20),
        translate=(0, 0.2),
        scale=(0.9, 1.2)
    ),
    transforms.ToTensor(),
    normalize,
])

In [ ]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files = None):
        super().__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [ ]:
# Form train/valid dataloaders
train_set = FoodDataset(os.path.join(cfg['dataset_root'],"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=0, pin_memory=True)

valid_set = FoodDataset(os.path.join(cfg['dataset_root'], "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

One /content/drive/MyDrive/food11-hw13/training sample /content/drive/MyDrive/food11-hw13/training/0_0.jpg
One /content/drive/MyDrive/food11-hw13/validation sample /content/drive/MyDrive/food11-hw13/validation/0_0.jpg


### Architecture_Design

In this homework, you have to design a smaller network and make it perform well. Apparently, a well-designed architecture is crucial for such task. Here, we introduce the depthwise and pointwise convolution. These variants of convolution are some common techniques for architecture design when it comes to network compression.

<img src="https://i.imgur.com/LFDKHOp.png" width=400px>

* explanation of depthwise and pointwise convolutions:
    * [prof. Hung-yi Lee's slides(p.24~p.30, especially p.28)](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/tiny_v7.pdf)

In [ ]:
# Example implementation of Depthwise and Pointwise Convolution
# def dwpw_conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
#     return nn.Sequential(
#         nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride, padding=padding, groups=in_channels), #depthwise convolution
#         nn.Conv2d(in_channels, out_channels, 1), # pointwise convolution
#     )
# 重新定义深度可分离卷积块（包含批归一化和激活函数）
def dwpw_conv(ic, oc, kernel_size=3, stride=2, padding=1):
    return nn.Sequential(
        # 深度卷积
        nn.Conv2d(ic, ic, kernel_size, stride=stride, padding=padding, groups=ic),
        nn.BatchNorm2d(ic),                    # 批归一化
        nn.LeakyReLU(0.01, inplace=True),      # LeakyReLU激活函数
        # 逐点卷积
        nn.Conv2d(ic, oc, 1),
        nn.BatchNorm2d(oc),                    # 批归一化
        nn.LeakyReLU(0.01, inplace=True)       # LeakyReLU激活函数
    )


* other useful techniques
    * [group convolution](https://www.researchgate.net/figure/The-transformations-within-a-layer-in-DenseNets-left-and-CondenseNets-at-training-time_fig2_321325862) (Actually, depthwise convolution is a specific type of group convolution)
    * [SqueezeNet](!https://arxiv.org/abs/1602.07360)
    * [MobileNet](!https://arxiv.org/abs/1704.04861)
    * [ShuffleNet](!https://arxiv.org/abs/1707.01083)
    * [Xception](!https://arxiv.org/abs/1610.02357)
    * [GhostNet](!https://arxiv.org/abs/1911.11907)


After introducing depthwise and pointwise convolutions, let's define the **student network architecture**. Here, we have a very simple network formed by some regular convolution layers and pooling layers. You can replace the regular convolution layers with the depthwise and pointwise convolutions. In this way, you can further increase the depth or the width of your network architecture.

In [ ]:
# 定义学生网络（轻量级模型）
class StudentNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 初始卷积层（类似ResNet的开始）
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 使用深度可分离卷积的层
        self.layer1 = dwpw_conv(64, 64, stride=1)
        self.layer2 = dwpw_conv(64, 128)
        self.layer3 = dwpw_conv(128, 256)
        self.layer4 = dwpw_conv(256, 140)

        # 全局平均池化和分类层
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(140, 11)                 # 11个食物类别

    def forward(self, x):
        # 前向传播
        out = self.relu(self.bn1(self.conv1(x)))    # 初始卷积+BN+ReLU
        out = self.maxpool(out)                     # 最大池化
        out = self.layer1(out)                      # 深度可分离卷积层1
        out = self.layer2(out)                      # 深度可分离卷积层2
        out = self.layer3(out)                      # 深度可分离卷积层3
        out = self.layer4(out)                      # 深度可分离卷积层4
        out = self.avgpool(out)                     # 全局平均池化
        out = out.flatten(1)                        # 展平
        out = self.dropout(out)                     # 防止过拟合
        out = self.fc(out)                          # 全连接层
        return out

# 创建学生模型的函数
def get_student_model():
    return StudentNet()

After specifying the student network architecture, please use `torchsummary` package to get information about the network and verify the total number of parameters. Note that the total params of your student network should not exceed the limit (`Total params` in `torchsummary` ≤ 100,000).

In [ ]:
# DO NOT modify this block and please make sure that this block can run sucessfully.
student_model = get_student_model()
summary(student_model, (3, 224, 224), device='cpu')
# You have to copy&paste the results of this block to HW13 GradeScope.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]             640
       BatchNorm2d-6           [-1, 64, 56, 56]             128
         LeakyReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]           4,160
       BatchNorm2d-9           [-1, 64, 56, 56]             128
        LeakyReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11           [-1, 64, 28, 28]             640
      BatchNorm2d-12           [-1, 64, 28, 28]             128
        LeakyReLU-13           [-1, 64, 28, 28]               0
           Conv2d-14          [-1, 128,

In [ ]:
# Load provided teacher model (model architecture: resnet18, num_classes=11, test-acc ~= 89.9%)
teacher_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=11)
# load state dict
teacher_ckpt_path = os.path.join(cfg['dataset_root'], "resnet18_teacher.ckpt")
teacher_model.load_state_dict(torch.load(teacher_ckpt_path, map_location='cpu'))
# 显示教师模型结构摘要
summary(teacher_model, (3, 224, 224), device='cpu')
# Now you already know the teacher model's architecture. You can take advantage of it if you want to pass the strong or boss baseline.
# Source code of resnet in pytorch: (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py)
# You can also see the summary of teacher model. There are 11,182,155 parameters totally in the teacher model
# summary(teacher_model, (3, 224, 224), device='cpu')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

### Knowledge_Distillation

<img src="https://i.imgur.com/H2aF7Rv.png=100x" width="400px">

Since we have a learned big model, let it teach the other small model. In implementation, let the training target be the prediction of big model instead of the ground truth.

**Why it works?**
* If the data is not clean, then the prediction of big model could ignore the noise of the data with wrong labeled.
* There might have some relations between classes, so soft labels from teacher model might be useful. For example, Number 8 is more similar to 6, 9, 0 than 1, 7.


**How to implement?**
* $Loss = \alpha T^2 \times KL(p || q) + (1-\alpha)(\text{Original Cross Entropy Loss}), \text{where } p=softmax(\frac{\text{student's logits}}{T}), \text{and } q=softmax(\frac{\text{teacher's logits}}{T})$
* very useful link: [pytorch docs of KLDivLoss with examples](!https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html)
* original paper: [Distilling the Knowledge in a Neural Network](!https://arxiv.org/abs/1503.02531)

In [ ]:
# 传统知识蒸馏->只看最终答案
# Hook知识蒸馏->看思考过程 学生也要在相应层学会看到这些特征(如边缘纹理等)

# 使用教师模型的预训练权重初始化学生模型的部分层
def use_pretrain():
    student_model.conv1.weight = teacher_model.conv1.weight
    student_model.bn1.weight = teacher_model.bn1.weight
    student_model.bn1.bias = teacher_model.bn1.bias
    student_model.bn1.running_mean = teacher_model.bn1.running_mean
    student_model.bn1.running_var = teacher_model.bn1.running_var

    # 冻结这些层的参数（不参与训练）
    student_model.conv1.weight.requires_grad = False
    student_model.bn1.weight.requires_grad = False
    student_model.bn1.bias.requires_grad = False

use_pretrain()  # 执行预训练权重转移

# 钩子工具类，用于提取中间层特征
class HookTool:
    def __init__(self):
        self.fea = None

    def hook_fun(self, module, fea_in, fea_out):
        self.fea = fea_out  # 保存输出特征

# 为指定层注册前向钩子的函数-安装"窃听器"
def get_feas_by_hook(model, names=['layer1', 'layer2', 'layer3']):
    fea_hooks = []
    for name, module in model.named_modules():
        if name in names:
            cur_hook = HookTool()
            module.register_forward_hook(cur_hook.hook_fun)  # 注册钩子
            fea_hooks.append(cur_hook)
    return fea_hooks

# 为教师和学生模型注册钩子
fea_hooks_teacher = get_feas_by_hook(teacher_model)
fea_hooks_student = get_feas_by_hook(student_model)

# 计算特征层之间的损失-比较中间思考过程
def loss_fea_layers(student, teacher):
    loss = 0
    for i in range(len(student)):
        # 加权损失：后面的层权重更大-比较学生和教师在第i层的"思考"是否相似
        loss += (len(student) - i) * F.smooth_l1_loss(student[i].fea, teacher[i].fea)
    return loss

In [ ]:
# Implement the loss function with KL divergence loss for knowledge distillation.
# You also have to copy-paste this whole block to HW13 GradeScope.
# def loss_fn_kd(student_logits, labels, teacher_logits, alpha=0.5, temperature=1.0):
#     pass

CE = nn.CrossEntropyLoss()  # 交叉熵损失

# 知识蒸馏损失函数
def loss_fn_kd(student_logits, labels, teacher_logits, alpha=0.5, temperature=20.0):
    # 软标签：使用温度参数软化logits
    student_T = (student_logits/temperature).softmax(dim=-1)
    teacher_T = (teacher_logits/temperature).softmax(dim=-1)

    # KL散度损失（知识蒸馏损失）
    kl_loss = (teacher_T*(teacher_T.log() - student_T.log())).sum(1).mean()

    # 交叉熵损失（硬标签损失）
    ce_loss = CE(student_logits, labels)

    # 组合损失：alpha控制两种损失的权重，temperature^2用于平衡梯度
    return alpha*(temperature**2)*kl_loss + (1 - alpha)*ce_loss

In [ ]:
# choose the loss function by the config
if cfg['loss_fn_type'] == 'CE':
    # For the classification task, we use cross-entropy as the default loss function.
    loss_fn = nn.CrossEntropyLoss() # loss function for simple baseline.

if cfg['loss_fn_type'] == 'KD': # KD stands for knowledge distillation
    loss_fn = loss_fn_kd # implement loss_fn_kd for the report question and the medium baseline.

# You can also adopt other types of knowledge distillation techniques for strong and boss baseline, but use function name other than `loss_fn_kd`
# For example:
# def loss_fn_custom_kd():
#     pass
# if cfg['loss_fn_type'] == 'custom_kd':
#     loss_fn = loss_fn_custom_kd

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
log(f"device: {device}")

# The number of training epochs and patience.
n_epochs = cfg['n_epochs']
patience = cfg['patience'] # If no improvement in 'patience' epochs, early stop

device: cuda


### Training
implement training loop for simple baseline, feel free to modify it.

In [ ]:
# 将模型移到计算设备
student_model.to(device)
teacher_model.to(device)

# 优化器：只优化需要梯度的参数
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, student_model.parameters()),
                            lr=cfg['lr'], weight_decay=cfg['weight_decay'])

# 学习率调度器：余弦退火重启
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=9, T_mult=2, eta_min=1e-5)

# 早停相关变量
stale = 0      # 连续没有改善的epoch数
best_acc = 0.0 # 最佳验证准确率

teacher_model.eval()  # 教师模型设为评估模式

# 训练循环
for epoch in range(n_epochs):
    student_model.train()  # 学生模型设为训练模式

    # 训练统计变量
    train_loss = []
    train_loss_fea = []
    train_accs = []
    train_lens = []

    # 计算当前epoch的进度百分比
    percent = (1+epoch)/n_epochs

    # 训练一个epoch
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)

        # 获取教师模型的输出（不计算梯度）
        with torch.no_grad():
            teacher_logits = teacher_model(imgs)

        # 学生模型前向传播
        logits = student_model(imgs)

        # 计算逻辑损失（知识蒸馏损失）
        # alpha随训练进度变化：开始时更依赖教师，后期更依赖真实标签
        loss_logits = loss_fn(logits, labels, teacher_logits, alpha=1 - percent*percent)

        # 计算特征损失
        loss_fea = loss_fea_layers(fea_hooks_student, fea_hooks_teacher)

        # 总损失：逻辑损失权重随训练进度增加，特征损失保持恒定
        loss = (10*percent*percent) * loss_logits + loss_fea

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=cfg['grad_norm_max'])
        optimizer.step()

        # 计算准确率
        acc = (logits.argmax(dim=-1) == labels).float().sum()
        train_batch_len = len(imgs)

        # 累积统计信息
        train_loss.append(loss_logits.item() * train_batch_len)
        train_loss_fea.append(loss_fea.item() * train_batch_len)
        train_accs.append(acc)
        train_lens.append(train_batch_len)

    # 计算训练指标
    train_loss = sum(train_loss) / sum(train_lens)
    train_loss_fea = sum(train_loss_fea) / sum(train_lens)
    train_acc = sum(train_accs) / sum(train_lens)

    log(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, loss_fea = {train_loss_fea:.5f}, acc = {train_acc:.5f}")

    # 验证阶段
    student_model.eval()
    valid_loss = []
    valid_accs = []
    valid_lens = []

    for batch in tqdm(valid_loader):
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)

        # 验证时不计算梯度
        with torch.no_grad():
            logits = student_model(imgs)
            teacher_logits = teacher_model(imgs)

        loss = loss_fn(logits, labels, teacher_logits)
        acc = (logits.argmax(dim=-1) == labels).float().sum()
        batch_len = len(imgs)

        valid_loss.append(loss.item() * batch_len)
        valid_accs.append(acc)
        valid_lens.append(batch_len)

    # 计算验证指标
    valid_loss = sum(valid_loss) / sum(valid_lens)
    valid_acc = sum(valid_accs) / sum(valid_lens)

    # 记录验证结果
    if valid_acc > best_acc:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} ---------------------> best")
    else:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    # 保存最佳模型和早停检查
    if valid_acc > best_acc:
        log(f"Best model found at epoch {epoch+1}, saving model")
        torch.save(student_model.state_dict(), f"{save_path}/student_best.ckpt")
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            log(f"No improvment {patience} consecutive epochs, early stopping")
            break

log("Finish training")
log_fw.close()

  0%|          | 0/155 [00:00<?, ?it/s]

[ Train | 001/150 ] loss = 18.65147, loss_fea = 3.00103, acc = 0.23870


  0%|          | 0/54 [00:00<?, ?it/s]

[ Valid | 001/150 ] loss = 16.69830, acc = 0.31157 ---------------------> best
Best model found at epoch 1, saving model


  0%|          | 0/155 [00:00<?, ?it/s]

...

[ Valid | 149/150 ] loss = 6.39277, acc = 0.79365


  0%|          | 0/155 [00:00<?, ?it/s]

[ Train | 150/150 ] loss = 0.55793, loss_fea = 1.89073, acc = 0.82029


  0%|          | 0/54 [00:00<?, ?it/s]

[ Valid | 150/150 ] loss = 6.60750, acc = 0.78898
Finish training


### Inference
load the best model of the experiment and generate submission.csv

In [ ]:
# create dataloader for evaluation
eval_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=test_tfm)
eval_loader = DataLoader(eval_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

One /content/drive/MyDrive/food11-hw13/evaluation sample /content/drive/MyDrive/food11-hw13/evaluation/0000.jpg


In [ ]:
# Load model from {exp_name}/student_best.ckpt
student_model_best = get_student_model() # get a new student model to avoid reference before assignment.
ckpt_path = f"{save_path}/student_best.ckpt" # the ckpt path of the best student model.
student_model_best.load_state_dict(torch.load(ckpt_path, map_location='cpu')) # load the state dict and set it to the student model
student_model_best.to(device) # set the student model to device

# Start evaluate
student_model_best.eval()
eval_preds = [] # storing predictions of the evaluation dataset

# Iterate the validation set by batches.
for batch in tqdm(eval_loader):
    # A batch consists of image data and corresponding labels.
    imgs, _ = batch
    # We don't need gradient in evaluation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = student_model_best(imgs.to(device))
        preds = list(logits.argmax(dim=-1).squeeze().cpu().numpy())
    # loss and acc can not be calculated because we do not have the true labels of the evaluation set.
    eval_preds += preds

def pad4(i):
    return "0"*(4-len(str(i))) + str(i)

# Save prediction results
ids = [pad4(i) for i in range(0,len(eval_set))]
categories = eval_preds

df = pd.DataFrame()
df['Id'] = ids
df['Category'] = categories
df.to_csv(f"{save_path}/submission.csv", index=False) # now you can download the submission.csv and upload it to the kaggle competition.

  0%|          | 0/53 [00:00<?, ?it/s]

> Don't forget to answer the report questions on GradeScope!